In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
#指定当前工作路径
import os
os.chdir("drive/Colab Notebooks/DataWhale")

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import preprocessing
# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns

In [0]:
data = pd.read_csv("data2.csv",engine = 'python',encoding="gbk")
data

In [0]:
y_data = data['status']
data.drop(['status'],axis = 1,inplace = True)
data

In [0]:
data.dtypes

In [0]:
#2 缺失值处理
def desc(df):
    loss_num=[] #缺失值的数量
    valid_num=[] #非缺失值的数量
    unique_num=[] #唯一的数量
    unique_percent=[] #唯一率
    for i,j in enumerate (df.columns): # enumerator 函数可以同时获得索引和值。 i表示索引，j表示值
        loss_num.append(df[df[j].isnull()].shape[0]) #isnull() 表示的是空 返回的是True（空）
        valid_num.append(df[df[j].notnull()].shape[0])
        unique_num.append(len(df[j].unique()))
        unique_percent.append(len(df[df[j].notnull()][j].unique())/df[df[j].notnull()].shape[0])
    data_desc = pd.DataFrame({"loss_num":loss_num,"valid_num":valid_num,"unique_num":unique_num,"unique_percent":unique_percent},index=df.columns)
    return data_desc
data_count = desc(data)
data_count

In [147]:
dict1 = data.isnull().sum()  #统计每列缺失值的方法，将统计结果保存到dict中
for key in list(dict1.keys()):
    if dict1[key] == 0:
        del dict1[key]
    elif dict1[key]>2000:
        print (key)
        del dict1[key]
dict1  #去掉无缺失值或者缺失值较多的列

Series([], dtype: int64)

In [0]:
data.drop(['student_feature'],axis = 1,inplace = True)

In [0]:
#处理数值型数据
for key in dict1.keys():
    if data[key].dtype != object:
        data[key].fillna(data[key].median(),inplace = True)
    else:
        print('object类型的缺失值和个数',key,dict1[key])
data.isnull().sum()

**分析object类型数据的缺失值**

In [0]:
data['reg_preference_for_trad'].value_counts() ##查看该列信息，根据信息填补缺失值

In [0]:
data['reg_preference_for_trad'].fillna("其他城市",inplace=True)

In [0]:
data.drop(['id_name'],axis = 1,inplace = True) #删减没有意义的列

In [0]:
#检查缺失值是否填充完全
dict1 = data.isnull().sum()
for key in list(dict1.keys()):
    if dict1[key]== 0:
        print("error")

**数据处理**

数据型数据归一化

In [148]:
min_max_scaler = preprocessing.MinMaxScaler()
for i in data.columns:
    if data[i].dtype == object:
        print(i)
    else:
        data[i] = (data[i]- data[i].min())/(data[i].max()-data[i].min())

trade_no
bank_card_no
reg_preference_for_trad
source
latest_query_time
loans_latest_time


In [0]:
data.head()


In [0]:
map_dict={'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他':0}
data['reg_preference_for_trad']=data['reg_preference_for_trad'].map(map_dict)
data.head()


In [0]:
data.columns

In [0]:
#drop_columns=['Unnamed: 0','bank_card_no','source','custid','trade_no',"trade_no",'latest_query_time',"loans_latest_time"]
#data.drop(drop_columns,axis=1,inplace=True)
data.drop('reg_preference_for_trad',axis=1,inplace=True)

In [167]:
#丢弃方差为0的列
data.drop(data.columns[data.std()==0],axis=1,inplace=True)
data.shape

(4754, 79)

In [168]:
dict1 = data.isnull().sum()  #统计每列缺失值的方法，将统计结果保存到dict中
for key in list(dict1.keys()):
    if dict1[key] == 0:
        del dict1[key]
    elif dict1[key]>2000:
        print (key)
        del dict1[key]
dict1

Series([], dtype: int64)

In [169]:
data_count = desc(data)
data_count

,loss_num,valid_num,unique_num,unique_percent
low_volume_percent,0,4754,40,0.008414
middle_volume_percent,0,4754,90,0.018931
take_amount_in_later_12_month_highest,0,4754,166,0.034918
trans_amount_increase_rate_lately,0,4754,782,0.164493
trans_activity_month,0,4754,84,0.017669
trans_activity_day,0,4754,512,0.107699
transd_mcc,0,4754,41,0.008624
trans_days_interval_filter,0,4754,147,0.030921
trans_days_interval,0,4754,114,0.023980
regional_mobility,0,4754,5,0.001052


In [171]:
#拆分测试集和训练集
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,y_data,train_size=0.7, random_state=2018)
print(x_train.shape,y_train.shape)

(3327, 79) (3327,)


In [172]:
from sklearn import ensemble
model_random_forest_Classifier = ensemble.RandomForestClassifier(n_estimators=20,random_state = 2019)
model_random_forest_Classifier.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=2019, verbose=0,
            warm_start=False)

In [0]:
# 性能评估
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, x_train, x_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(x_train)
    y_test_pred = clf.predict(x_test)
    
    y_train_proba = clf.predict_proba(x_train)[:,1]
    y_test_proba = clf.predict_proba(x_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

In [174]:
#构架模型并评价
from sklearn.linear_model import LogisticRegressionCV

clf=LogisticRegressionCV(class_weight='balanced',max_iter=5000)
clf.fit(x_train,y_train)
model_metrics(clf, x_train, x_test, y_train, y_test)

[准确率] 训练集： 0.7656 测试集： 0.7155
[auc值] 训练集： 0.8221 测试集： 0.7679


在机器学习的二分类问题中，IV值（Information Value）主要用来对输入变量进行编码和预测能力评估。特征变量IV值的大小即表示该变量预测能力的强弱。IV 值的取值范围是 [0, 正无穷)，如果当前分组中只包含响应客户或者未响应客户时，IV = 正无穷。

In [0]:
import math
from scipy import stats
from sklearn.utils.multiclass import type_of_target

def woe(X, y, event=1):  
    res_woe = []
    iv_dict = {}
    for feature in X.columns:
        x = X[feature].values
        # 1) 连续特征离散化
        if type_of_target(x) == 'continuous':
            x = discrete(x)
        # 2) 计算该特征的woe和iv
        # woe_dict, iv = woe_single_x(x, y, feature, event)
        woe_dict, iv = woe_single_x(x, y, feature, event)
        iv_dict[feature] = iv
        res_woe.append(woe_dict) 
        
    return iv_dict
        
def discrete(x):
    # 使用5等分离散化特征
    res = np.zeros(x.shape)
    for i in range(5):
        point1 = stats.scoreatpercentile(x, i * 20)
        point2 = stats.scoreatpercentile(x, (i + 1) * 20)
        x1 = x[np.where((x >= point1) & (x <= point2))]
        mask = np.in1d(x, x1)
        res[mask] = i + 1    # 将[i, i+1]块内的值标记成i+1
    return res

def woe_single_x(x, y, feature,event = 1):
    # event代表预测正例的标签
    event_total = sum(y == event)
    non_event_total = y.shape[-1] - event_total
    
    iv = 0
    woe_dict = {}
    for x1 in set(x):    # 遍历各个块
        y1 = y.reindex(np.where(x == x1)[0])
        event_count = sum(y1 == event)
        non_event_count = y1.shape[-1] - event_count
        rate_event = event_count / event_total    
        rate_non_event = non_event_count / non_event_total
        
        if rate_event == 0:
            rate_event = 0.0001
            # woei = -20
        elif rate_non_event == 0:
            rate_non_event = 0.0001
            # woei = 20
        woei = math.log(rate_event / rate_non_event)
        woe_dict[x1] = woei
        iv += (rate_event - rate_non_event) * woei
    return woe_dict, iv

In [176]:
iv_dict = woe(x_train, y_train)
iv = sorted(iv_dict.items(), key = lambda x:x[1],reverse = True)
iv

[('history_fail_fee', 0.2380638126934534),
 ('latest_six_month_apply', 0.2336559521090925),
 ('latest_query_day', 0.23363892704251113),
 ('trans_days_interval', 0.2310974133469704),
 ('trans_activity_day', 0.23089021521474926),
 ('avg_price_last_12_month', 0.23076797912604352),
 ('trans_amount_3_month', 0.23010632257866417),
 ('loans_avg_limit', 0.22937233933022275),
 ('loans_credibility_limit', 0.22916410940818902),
 ('apply_score', 0.22883669416196517),
 ('low_volume_percent', 0.22857880126543986),
 ('history_suc_fee', 0.2284258397510279),
 ('avg_price_top_last_12_valid_month', 0.22804418697211443),
 ('historical_trans_day', 0.22785892580201067),
 ('consfin_credibility', 0.22616105192402294),
 ('latest_one_month_loan', 0.2259858987958161),
 ('first_transaction_day', 0.22577667440590374),
 ('trans_day_last_12_month', 0.22575246522750658),
 ('trans_amount_increase_rate_lately', 0.2256873870490378),
 ('middle_volume_percent', 0.22562502839194543),
 ('first_transaction_time', 0.225580293

In [0]:
threshold = 0.1
data_index = []
for i in range(len(iv)):
    if iv[i][1] < threshold:
        data_index.append(iv[i])
        print(iv[i])
#X_train.drop(data_index, axis=1, inplace=True)

In [179]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=120, max_depth=9, min_samples_split=50,
                            min_samples_leaf=20, max_features = 9,oob_score=True, random_state=2333)
rf.fit(x_train, y_train)
print('袋外分数：', rf.oob_score_)
model_metrics(rf, x_train, x_test, y_train, y_test)
feature_importance1 = sorted(zip(map(lambda x: '%.4f'%x, rf.feature_importances_), list(x_train.columns)), reverse=True)

袋外分数： 0.7917042380522994
[准确率] 训练集： 0.8209 测试集： 0.7856
[auc值] 训练集： 0.8991 测试集： 0.7729


In [0]:
feature_importance1

In [0]:
useless=[]
for feature in x_train.columns:
    if feature in [t[1] for t in feature_importance1[30:]]:
        useless.append(feature)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,roc_curve

lr_model = LogisticRegression(C = 0.1, penalty = 'l1')
svm_model = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
dt_model = DecisionTreeClassifier(max_depth=5,min_samples_split=50,min_samples_leaf=60, max_features=9, random_state =2333)
xgb_model = XGBClassifier(learning_rate =0.1, n_estimators=80, max_depth=3, min_child_weight=5, 
                    gamma=0.2, subsample=0.8, colsample_bytree=0.8, reg_alpha=1e-5, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
gbdt_model=GradientBoostingClassifier(n_estimators=100)

models={'LR':lr_model, 'SVM':svm_model, 'DT':dt_model, 'GBDT':gbdt_model, 'XGBoost':xgb_model}

In [186]:
df_result=pd.DataFrame(columns=('model','accuracy','precision','recall','f1_score','auc'))
row=0
def evaluate(y_pre,y):
    acc=accuracy_score(y,y_pre)
    p=precision_score(y,y_pre)
    r=recall_score(y,y_pre)
    f1=f1_score(y,y_pre)
    return acc,p,r,f1

for name,model in models.items():
    print(name,'start training...')
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    y_proba=model.predict_proba(X_test)
    acc,p,r,f1=evaluate(y_pred,y_test)
    auc=roc_auc_score(y_test,y_proba[:,1])
    df_result.loc[row]=[name,acc,p,r,f1,auc]
    row+=1
print(df_result)

LR start training...
SVM start training...
DT start training...
GBDT start training...
XGBoost start training...
     model  accuracy  precision    recall  f1_score       auc
0       LR  0.787666   0.672840  0.303621  0.418426  0.779256
1      SVM  0.778556   0.700935  0.208914  0.321888  0.780398
2       DT  0.753329   0.524476  0.208914  0.298805  0.714894
3     GBDT  0.775753   0.590698  0.353760  0.442509  0.767076
4  XGBoost  0.789769   0.649746  0.356546  0.460432  0.775263
